<a href="https://colab.research.google.com/github/buwituze/pre-consultation-agent/blob/main/speech_to_text_asr_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️ Speech-to-Text Module — Kinyarwanda & English ASR

**Purpose:** Convert patient speech (Kinyarwanda or English) into accurate transcribed text with confidence scores.

| | |
|---|---|
| **Input** | Raw audio (`.wav`, `.mp3`, `.flac`, `.m4a`) |
| **Output** | Transcribed text + confidence score per segment |
| **Kinyarwanda Model** | `akera/whisper-large-v3-kin-200h-v2` |
| **English Model** | `openai/whisper-large-v3` |

### Pipeline Overview
```
Audio Input
    │
    ▼
Language Detection  ──►  Kinyarwanda  ──►  Akera Whisper Model
    │                                            │
    └──────────────►  English       ──►  OpenAI Whisper Model
                                                 │
                                    Transcription + Confidence Score
```

> ⚠️ **Runtime:** Set to **GPU (T4 or better)** via `Runtime > Change runtime type` for best performance.

---
## 📦 Section 1 — Install Dependencies

In [ ]:
# Install all required libraries
!pip install -q -U transformers accelerate
!pip install -q langdetect librosa soundfile torchaudio
!pip install -q datasets  # optional: for loading test audio

print("✅ Dependencies installed.")

---
## 🔧 Section 2 — Imports & Device Setup

In [ ]:
import torch
import numpy as np
import librosa
import soundfile as sf
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
from langdetect import detect, LangDetectException
from dataclasses import dataclass
from typing import Optional
import warnings
warnings.filterwarnings("ignore")

# ── Device configuration ──────────────────────────────────────────────────────
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TORCH_DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32

print(f"🖥️  Device  : {DEVICE.upper()}")
if DEVICE == "cuda":
    print(f"🎮  GPU     : {torch.cuda.get_device_name(0)}")
    print(f"💾  VRAM    : {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected — inference will be slow on CPU.")

---
## 🤖 Section 3 — Load ASR Models

Both models are loaded once and cached. Loading takes ~2–5 minutes on first run (downloading weights).

In [ ]:
# ── Model identifiers ─────────────────────────────────────────────────────────
KIN_MODEL_ID = "akera/whisper-large-v3-kin-200h-v2"   # Kinyarwanda
ENG_MODEL_ID = "openai/whisper-large-v3"               # English

TARGET_SAMPLE_RATE = 16_000  # Whisper expects 16 kHz audio

def load_whisper_pipeline(model_id: str, language: Optional[str] = None) -> pipeline:
    """Load a Whisper model as a HuggingFace ASR pipeline."""
    print(f"  Loading: {model_id} ...")
    pipe = pipeline(
        task="automatic-speech-recognition",
        model=model_id,
        torch_dtype=TORCH_DTYPE,
        device=DEVICE,
        # Return per-token log probabilities for confidence scoring
        return_timestamps=True,
    )
    # Pin forced language decoder token if specified
    if language:
        pipe.model.config.forced_decoder_ids = (
            pipe.tokenizer.get_decoder_prompt_ids(language=language, task="transcribe")
        )
    print(f"  ✅ Loaded: {model_id}")
    return pipe


print("\n🔄 Loading Kinyarwanda model...")
kin_pipe = load_whisper_pipeline(KIN_MODEL_ID)

print("\n🔄 Loading English model...")
eng_pipe = load_whisper_pipeline(ENG_MODEL_ID, language="english")

print("\n🎉 Both models ready!")

---
## 🌐 Section 4 — Language Detection

A lightweight text-based detection step identifies the dominant language of each audio segment **after** a quick Whisper decode pass. This avoids a separate audio-level classifier and leverages Whisper's multilingual capability.

In [ ]:
# ── Supported language codes ──────────────────────────────────────────────────
# langdetect uses ISO 639-1 codes; Kinyarwanda = 'rw'
KINYARWANDA_CODES = {"rw"}        # ISO codes that map to Kinyarwanda
ENGLISH_CODES     = {"en"}        # ISO codes that map to English


def detect_language_from_text(text: str) -> str:
    """
    Detect dominant language from transcribed text.

    Returns:
        'kinyarwanda' | 'english' | 'unknown'
    """
    if not text or len(text.strip()) < 5:
        return "unknown"
    try:
        lang_code = detect(text)
        if lang_code in KINYARWANDA_CODES:
            return "kinyarwanda"
        elif lang_code in ENGLISH_CODES:
            return "english"
        else:
            return "unknown"
    except LangDetectException:
        return "unknown"


def detect_language_from_audio(audio_array: np.ndarray, sample_rate: int) -> str:
    """
    Detect language by doing a quick decode with the English Whisper model
    and reading its internal language prediction token.

    Falls back to text-based detection if token is unavailable.
    """
    # Use the English (multilingual) model for language ID
    result = eng_pipe(
        {"array": audio_array, "sampling_rate": sample_rate},
        generate_kwargs={"task": "transcribe", "language": None},  # let model decide
        return_timestamps=False,
    )
    detected_text = result.get("text", "")
    return detect_language_from_text(detected_text)


print("✅ Language detection functions defined.")

---
## 📊 Section 5 — Confidence Scoring

In [ ]:
def compute_confidence_from_chunks(chunks: list) -> float:
    """
    Derive an aggregate confidence score (0.0 – 1.0) from Whisper
    timestamp chunks.

    Whisper doesn't expose token-level log-probs through the pipeline directly,
    so we approximate confidence using segment-level heuristics:
      - Presence and count of clean timestamp chunks
      - Absence of [BLANK_AUDIO] / repetition artifacts

    For production, replace with log-prob extraction from model.generate().
    """
    if not chunks:
        return 0.0

    penalties = 0.0
    for chunk in chunks:
        text = chunk.get("text", "")
        # Common low-quality Whisper outputs
        if any(tag in text for tag in ["[BLANK_AUDIO]", "[MUSIC]", "♪"]):
            penalties += 0.3
        # Detect repetition (hallucination signal)
        words = text.strip().split()
        if len(words) > 2 and len(set(words)) / len(words) < 0.5:
            penalties += 0.2

    raw_score = max(0.0, 1.0 - (penalties / max(len(chunks), 1)))
    return round(raw_score, 3)


print("✅ Confidence scoring function defined.")

---
## 🔊 Section 6 — Audio Preprocessing

In [ ]:
def load_and_preprocess_audio(audio_path: str, target_sr: int = TARGET_SAMPLE_RATE) -> tuple:
    """
    Load an audio file, resample to target sample rate, and convert to mono.

    Args:
        audio_path : Path to audio file (.wav / .mp3 / .flac / .m4a)
        target_sr  : Target sample rate (default 16 kHz for Whisper)

    Returns:
        (audio_array: np.ndarray, sample_rate: int)
    """
    audio_array, sr = librosa.load(audio_path, sr=target_sr, mono=True)
    # Normalize amplitude to [-1, 1]
    if audio_array.max() > 0:
        audio_array = audio_array / np.abs(audio_array).max()
    print(f"  📁 File    : {audio_path}")
    print(f"  ⏱️  Duration: {len(audio_array)/sr:.2f}s  |  Sample rate: {sr} Hz")
    return audio_array, sr


def split_audio_into_segments(audio_array: np.ndarray,
                               sample_rate: int,
                               segment_duration_s: float = 30.0) -> list:
    """
    Split audio into fixed-length segments for per-segment language detection.
    Whisper performs best on ≤30s chunks.

    Returns:
        List of (start_time, end_time, audio_chunk) tuples.
    """
    segment_len = int(segment_duration_s * sample_rate)
    segments = []
    for i, start in enumerate(range(0, len(audio_array), segment_len)):
        chunk = audio_array[start : start + segment_len]
        start_t = start / sample_rate
        end_t   = min((start + segment_len) / sample_rate,
                      len(audio_array) / sample_rate)
        segments.append((start_t, end_t, chunk))
    print(f"  🔪 Split into {len(segments)} segment(s) of ≤{segment_duration_s}s each.")
    return segments


print("✅ Audio preprocessing functions defined.")

---
## 🚀 Section 7 — Core Transcription Pipeline

In [ ]:
@dataclass
class SegmentResult:
    """Holds the result for a single audio segment."""
    start_time  : float
    end_time    : float
    language    : str
    text        : str
    confidence  : float
    model_used  : str


def transcribe_segment(
    audio_chunk  : np.ndarray,
    sample_rate  : int,
    forced_lang  : Optional[str] = None,
) -> dict:
    """
    Transcribe a single audio segment using the appropriate model.

    Args:
        audio_chunk : 1-D numpy array of audio samples
        sample_rate : Sample rate of the audio
        forced_lang : 'kinyarwanda' | 'english' | None (auto-detect)

    Returns:
        dict with keys: text, language, confidence, model_used, chunks
    """
    audio_input = {"array": audio_chunk, "sampling_rate": sample_rate}

    # ── Step 1: Language detection ────────────────────────────────────────────
    if forced_lang:
        detected_lang = forced_lang
    else:
        detected_lang = detect_language_from_audio(audio_chunk, sample_rate)
        if detected_lang == "unknown":
            # Default to Kinyarwanda for this medical domain (primary patient language)
            detected_lang = "kinyarwanda"
            print(f"  ⚠️  Language unclear → defaulting to Kinyarwanda")

    # ── Step 2: Route to appropriate model ───────────────────────────────────
    if detected_lang == "kinyarwanda":
        active_pipe = kin_pipe
        model_name  = KIN_MODEL_ID
        gen_kwargs  = {"task": "transcribe"}
    else:  # english
        active_pipe = eng_pipe
        model_name  = ENG_MODEL_ID
        gen_kwargs  = {"task": "transcribe", "language": "english"}

    # ── Step 3: Transcribe ────────────────────────────────────────────────────
    result = active_pipe(
        audio_input,
        generate_kwargs=gen_kwargs,
        return_timestamps=True,
    )

    text   = result.get("text", "").strip()
    chunks = result.get("chunks", [])
    conf   = compute_confidence_from_chunks(chunks)

    return {
        "text"       : text,
        "language"   : detected_lang,
        "confidence" : conf,
        "model_used" : model_name,
        "chunks"     : chunks,
    }


def transcribe_audio_file(
    audio_path       : str,
    forced_lang      : Optional[str] = None,
    segment_duration : float = 30.0,
    verbose          : bool = True,
) -> dict:
    """
    Full pipeline: load → segment → detect language → transcribe → aggregate.

    Args:
        audio_path       : Path to the audio file.
        forced_lang      : Override language detection ('kinyarwanda' or 'english').
        segment_duration : Duration of each segment in seconds (max 30 for Whisper).
        verbose          : Print progress logs.

    Returns:
        {
          'full_text'         : str,          # concatenated transcript
          'mean_confidence'   : float,        # average confidence score
          'dominant_language' : str,
          'segments'          : [SegmentResult, ...]
        }
    """
    if verbose:
        print("\n" + "═" * 60)
        print(" 🎙️  TRANSCRIPTION PIPELINE")
        print("═" * 60)

    # 1. Load & preprocess
    audio_array, sr = load_and_preprocess_audio(audio_path)

    # 2. Split into segments
    segments_raw = split_audio_into_segments(audio_array, sr, segment_duration)

    # 3. Transcribe each segment
    segment_results = []
    for idx, (start_t, end_t, chunk) in enumerate(segments_raw):
        if verbose:
            print(f"\n  ▶ Segment {idx+1}/{len(segments_raw)} "
                  f"[{start_t:.1f}s → {end_t:.1f}s]")

        seg_out = transcribe_segment(chunk, sr, forced_lang=forced_lang)

        sr_obj = SegmentResult(
            start_time = start_t,
            end_time   = end_t,
            language   = seg_out["language"],
            text       = seg_out["text"],
            confidence = seg_out["confidence"],
            model_used = seg_out["model_used"],
        )
        segment_results.append(sr_obj)

        if verbose:
            print(f"     Language   : {sr_obj.language}")
            print(f"     Model      : {sr_obj.model_used.split('/')[-1]}")
            print(f"     Confidence : {sr_obj.confidence:.3f}")
            print(f"     Text       : {sr_obj.text[:120]}{'...' if len(sr_obj.text)>120 else ''}")

    # 4. Aggregate results
    full_text  = " ".join(s.text for s in segment_results if s.text)
    mean_conf  = np.mean([s.confidence for s in segment_results]) if segment_results else 0.0
    lang_votes = [s.language for s in segment_results]
    dominant   = max(set(lang_votes), key=lang_votes.count) if lang_votes else "unknown"

    if verbose:
        print("\n" + "═" * 60)
        print(" ✅  TRANSCRIPTION COMPLETE")
        print("═" * 60)
        print(f"  Dominant Language : {dominant}")
        print(f"  Mean Confidence   : {mean_conf:.3f}")
        print(f"  Full Transcript   :\n\n  {full_text}\n")

    return {
        "full_text"         : full_text,
        "mean_confidence"   : round(float(mean_conf), 3),
        "dominant_language" : dominant,
        "segments"          : segment_results,
    }


print("✅ Transcription pipeline defined.")

---
## 🧪 Section 8 — Run Inference

### Option A — Transcribe an uploaded file

In [ ]:
# ── Option A: Upload your own audio file ─────────────────────────────────────
from google.colab import files

print("📂 Please upload an audio file (.wav / .mp3 / .flac / .m4a):")
uploaded = files.upload()

if uploaded:
    audio_filename = list(uploaded.keys())[0]
    result = transcribe_audio_file(
        audio_path   = audio_filename,
        forced_lang  = None,   # None = auto-detect | 'kinyarwanda' | 'english'
        segment_duration = 30.0,
        verbose      = True,
    )

### Option B — Transcribe a local path (e.g. Google Drive)

In [ ]:
# ── Option B: Mount Google Drive and provide a path ───────────────────────────
# from google.colab import drive
# drive.mount('/content/drive')
#
# audio_path = "/content/drive/MyDrive/patient_audio/sample.wav"
# result = transcribe_audio_file(audio_path, verbose=True)

### Option C — Test with a sample audio from HuggingFace datasets

In [ ]:
# ── Option C: Quick smoke-test using a public dataset sample ──────────────────
from datasets import load_dataset

print("🔄 Loading a sample from the LibriSpeech test-clean dataset...")
ds = load_dataset("hf-internal-testing/librispeech_asr_demo",
                  "clean", split="validation", trust_remote_code=True)

sample     = ds[0]
audio_arr  = np.array(sample["audio"]["array"], dtype=np.float32)
sample_sr  = sample["audio"]["sampling_rate"]

print(f"  Reference text: {sample['text']}\n")

# Run transcription directly on the numpy array (skip file load)
seg_out = transcribe_segment(audio_arr, sample_sr, forced_lang="english")

print("\n📝 Transcription Results:")
print(f"  Language   : {seg_out['language']}")
print(f"  Model      : {seg_out['model_used'].split('/')[-1]}")
print(f"  Confidence : {seg_out['confidence']:.3f}")
print(f"  Text       : {seg_out['text']}")

---
## 📋 Section 9 — Inspect Results & Export

In [ ]:
# ── View segment-by-segment breakdown ────────────────────────────────────────
# Run this cell after Option A or B above

try:
    print(f"\n{'─'*60}")
    print(f" SEGMENT BREAKDOWN ({len(result['segments'])} segment(s))")
    print(f"{'─'*60}")
    for i, seg in enumerate(result["segments"]):
        print(f"\n  [{i+1}] {seg.start_time:.1f}s → {seg.end_time:.1f}s")
        print(f"       Language   : {seg.language}")
        print(f"       Confidence : {seg.confidence:.3f}")
        print(f"       Model      : {seg.model_used.split('/')[-1]}")
        print(f"       Text       : {seg.text}")

    print(f"\n{'─'*60}")
    print(f" FULL TRANSCRIPT")
    print(f"{'─'*60}")
    print(result["full_text"])
    print(f"\n  Mean Confidence : {result['mean_confidence']:.3f}")
    print(f"  Dominant Language: {result['dominant_language']}")

except NameError:
    print("⚠️  No result found — please run Section 8 first.")

In [ ]:
# ── Export transcript to a JSON file ─────────────────────────────────────────
import json, datetime

try:
    export_data = {
        "timestamp"         : datetime.datetime.now().isoformat(),
        "dominant_language" : result["dominant_language"],
        "mean_confidence"   : result["mean_confidence"],
        "full_text"         : result["full_text"],
        "segments": [
            {
                "segment"    : i + 1,
                "start_time" : seg.start_time,
                "end_time"   : seg.end_time,
                "language"   : seg.language,
                "confidence" : seg.confidence,
                "model_used" : seg.model_used,
                "text"       : seg.text,
            }
            for i, seg in enumerate(result["segments"])
        ],
    }

    output_path = "transcript_output.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Transcript saved to: {output_path}")

    # Download to local machine
    from google.colab import files
    files.download(output_path)

except NameError:
    print("⚠️  No result found — please run Section 8 first.")

---
## 🔁 Section 10 — Batch Transcription (Multiple Files)

In [ ]:
def batch_transcribe(audio_paths: list, forced_lang: Optional[str] = None) -> list:
    """
    Transcribe a list of audio files and return all results.

    Args:
        audio_paths : List of file paths to audio files.
        forced_lang : Optional language override for all files.

    Returns:
        List of result dicts (same structure as transcribe_audio_file).
    """
    all_results = []
    for idx, path in enumerate(audio_paths):
        print(f"\n📄 File {idx+1}/{len(audio_paths)}: {path}")
        try:
            res = transcribe_audio_file(path, forced_lang=forced_lang, verbose=True)
            res["file"] = path
            all_results.append(res)
        except Exception as e:
            print(f"  ❌ Error: {e}")
            all_results.append({"file": path, "error": str(e)})
    return all_results


# Example usage — uncomment and adjust paths:
# batch_results = batch_transcribe(
#     audio_paths = ["patient_01.wav", "patient_02.wav", "patient_03.mp3"],
#     forced_lang = None,  # auto-detect
# )

print("✅ Batch transcription function defined.")

---
## 📝 Notes & Known Limitations

| Topic | Detail |
|---|---|
| **Confidence scores** | Approximated from segment heuristics. For token-level log-probs, use `model.generate()` with `return_dict_in_generate=True`. |
| **Code-switching** | Audio with heavy Kinyarwanda–English mixing may be routed to a single model. Consider splitting at silence boundaries. |
| **Background noise** | Whisper large-v3 is robust to light noise. Heavy noise may require a denoising pre-step (e.g., `noisereduce` library). |
| **Memory** | Loading both models simultaneously requires ~10–14 GB VRAM. If OOM errors occur, load models one at a time. |
| **Medical vocabulary** | No domain-adapted vocabulary. Downstream correction of medical terms is recommended before use. |
| **Real-time use** | This notebook is batch-oriented. For streaming, integrate with `pyaudio` + chunked inference. |